In [1]:
import pandas as pd 

In [2]:
df = pd.read_csv('datasets/loan_approval_dataset.csv')


In [3]:
print(df)


      loan_id   no_of_dependents      education  self_employed   income_annum  \
0           1                  2       Graduate             No        9600000   
1           2                  0   Not Graduate            Yes        4100000   
2           3                  3       Graduate             No        9100000   
3           4                  3       Graduate             No        8200000   
4           5                  5   Not Graduate            Yes        9800000   
...       ...                ...            ...            ...            ...   
4264     4265                  5       Graduate            Yes        1000000   
4265     4266                  0   Not Graduate            Yes        3300000   
4266     4267                  2   Not Graduate             No        6500000   
4267     4268                  1   Not Graduate             No        4100000   
4268     4269                  1       Graduate             No        9200000   

       loan_amount   loan_t

In [4]:
from rewrite import create_model
import argparse 

def get_label(model_str, tokenizer, pipe, sent, device):
    print(1)
    '''
    helper for predict function
    '''
    input_text = "Return the correct Loan label on the following people: \"" + sent + "\"Return Approved or Rejected. Answer: "
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)
    if model_str  == "google/flan-t5-large":
        outputs = pipe.generate(input_ids, max_length=3, early_stopping=True)
        out = tokenizer.decode(outputs[0])
    elif model_str == "google/flan-ul2":
        outputs = pipe.generate(input_ids, max_length=3, early_stopping=True)
        out = tokenizer.decode(outputs[0])
    elif model_str == "llama" or model_str == "llama-13b":
        outputs = pipe.generate(input_ids, temperature=0.1, max_new_tokens=3, early_stopping=True)
        out = tokenizer.decode(outputs[0], skip_special_tokens=True)
    elif model_str == "llama2" or model_str.lower() == "llama2-13b" or model_str.lower() == "llama2-7b":
        outputs = pipe.generate(input_ids, temperature=0.1, top_p = 0.1, max_new_tokens=3, early_stopping=True)
        out = tokenizer.decode(outputs[0], skip_special_tokens=True)
    elif model_str == "vicuna-13b":
        outputs = pipe.generate(input_ids, temperature=0.1, top_p = 50, max_new_tokens=3, early_stopping=True)
        out = tokenizer.decode(outputs[0], skip_special_tokens=True)
    elif model_str == "databricks/dolly-v1-6b":
        outputs = pipe.generate(input_ids, pad_token_id = tokenizer.eos_token_id, temperature=0.1, max_new_tokens=3, early_stopping=True)
        out = tokenizer.decode(outputs[0], skip_special_tokens=True)
    elif model_str == "cerebras/Cerebras-GPT-13B":
        outputs = pipe.generate(input_ids, pad_token_id = tokenizer.eos_token_id, temperature=0.1, max_new_tokens=3, early_stopping=True)
        out = tokenizer.decode(outputs[0], skip_special_tokens=True)
    elif model_str == "phi-2":
        outputs = pipe.generate(input_ids, max_new_tokens=30)
        out = tokenizer.batch_decode(outputs)[0]
    elif model_str == "gemma-7b-it":
        outputs = pipe.generate(input_ids, max_new_tokens=30)
        out = tokenizer.decode(outputs[0])
    else:
        print("not implemented")
    print(out)

    return extract_label(model_str, out)

def extract_label(model_str, p):
    print(p)
    if "google" in model_str:
        if "negative" in p:
            l = 0
        elif "positive" in p:
            l = 1
        else:
            l = -1
    else:
        if "Rejected" in p.lower()[-15:]:
            l = 0
        elif "Approved" in p.lower()[-15:]:
            l = 1
        else:
            l = -1    
    return l

def predict(model_str,tokenizer, pipe, sent_list, device):
    predicted_labels = []
    for sent in sent_list:
        predicted_labels.append(get_label(model_str, tokenizer, pipe, sent, device))
    return predicted_labels

def calculate_accuracy(predicted_labels, actual_labels):
        # Calculate accuracy
        print(len(predicted_labels),len(actual_labels))
        assert len(predicted_labels) == len(actual_labels), print("predicted_labels and actual_labels are not of the same length.")
        correct_predictions = sum(1 for pred, actual in zip(predicted_labels, actual_labels) if pred == actual)
        total_predictions = len(predicted_labels)
        accuracy = correct_predictions / total_predictions
        return accuracy

def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--seed_sentence', type=str, default="I used to think this is good but now I think it is bad")
    parser.add_argument('--n', type=int, default=10) ### how many sentences to generate 
    parser.add_argument('--k', type=int, default=5) ### 
    parser.add_argument('--model', type=str, default="llama2")
    parser.add_argument('--device',type = int, default = 1)
    parser.add_argument('--model2', type=str, default="llama2")
    args = parser.parse_args()
    return args


In [5]:
import pandas as pd
import torch
from sklearn.metrics import accuracy_score

# Load the dataset
data = df  # Replace with the actual dataset path

# Fill NaN values for simplicity
data.fillna("unknown", inplace=True)


In [6]:
data.columns

Index(['loan_id', ' no_of_dependents', ' education', ' self_employed',
       ' income_annum', ' loan_amount', ' loan_term', ' cibil_score',
       ' residential_assets_value', ' commercial_assets_value',
       ' luxury_assets_value', ' bank_asset_value', ' loan_status'],
      dtype='object')

In [7]:
import torch
import time
from rewrite import *
from predict import *
from enhance import *
from datasets import load_dataset
import pickle
from random import sample


In [9]:
# Function to encode each row as a text prompt for loan status prediction
data.columns = data.columns.str.strip()

# Fill NaN values for simplicity
data.fillna("unknown", inplace=True)

# Function to encode each row as a text prompt for loan status prediction
def encode_as_prompt(row):
    return (
        f"Suppose there is an applicant with {row['no_of_dependents']} dependents, "
        f"who is a {'graduate' if row['education'] == 'Graduate' else 'not graduate'}, "
        f"and {'self-employed' if row['self_employed'] == 'Yes' else 'not self-employed'}. "
        f"Their income is {row['income_annum']} dollars, and the requested loan amount is {row['loan_amount']} dollars, "
        f"with a loan term of {row['loan_term']} months. "
        f"The applicant has a CIBIL score of {row['cibil_score']}, residential assets worth {row['residential_assets_value']} dollars, "
        f"commercial assets worth {row['commercial_assets_value']} dollars, luxury assets worth {row['luxury_assets_value']} dollars, "
        f"and bank assets worth {row['bank_asset_value']} dollars. "
    )

# Apply the encoding function to each row
data['Prompt'] = data.apply(encode_as_prompt, axis=1)


# Assume the create_model and predict functions are defined and imported from your model library
device = 0
model_str = "phi-2"  # Replace with your desired model
tokenizer, pipe = create_model(model_str, 0)

# Predict the loan status using the LLM
predicted_labels = predict(model_str, tokenizer, pipe, data['Prompt'].tolist(), device)

# Convert labels from 1/0 to 'Approved'/'Rejected' if needed
predicted_loan_status = ['Approved' if label == 1 else 'Rejected' for label in predicted_labels]

# Add predictions to the dataframe
data['Predicted_Loan_Status'] = predicted_loan_status

# Calculate accuracy
accuracy = accuracy_score(data['loan_status'], data['Predicted_Loan_Status'])

# Print the number of correctly classified cases
print(f"Number of correctly classified cases: {len(data[data['loan_status'] == data['Predicted_Loan_Status']])}")
print(f"Total number of cases: {len(data)}")
print(f"Accuracy: {accuracy * 100:.2f}%")

# Optional: Display the first few rows to see predictions vs actuals
print(data[['loan_status', 'Predicted_Loan_Status']].head(10))


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

KeyboardInterrupt: 